## Kaggle

**Import**

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from scipy import stats

In [2]:
# Import datasets
X_train = pd.read_csv('../Datasets/train_X.csv')
y_train = pd.read_csv('../Datasets/train_y.csv')
X_test = pd.read_csv('../Datasets/public_private_X.csv')

In [3]:
# Import datasets again as a copy
X_train_copy = pd.read_csv('../Datasets/train_X.csv')
y_train_copy = pd.read_csv('../Datasets/train_y.csv')
X_test_copy = pd.read_csv('../Datasets/public_private_X.csv')

**EDA**

In [4]:
# Examine the training data
display(X_train.head(2))
print(X_train.columns)
print(X_train.shape)
display(y_train.head(2))
print(y_train.shape)

,ID,DIVISION_NUMBER,PRODUCT_NUMBER,PURCHASE_ORDER_DUE_DATE,COMPANY_VENDOR_NUMBER,SHIP_FROM_VENDOR,ORDER_DATE,ORDER_DAY_OF_WEEK,PRODUCT_CLASSIFICATION,PURCHASE_ORDER_TYPE,...,GIVEN_TIME_TO_LEAD_TIME_RATIO,DUE_DATE_WEEKDAY,PRODUCT_MARKET,RESERVABLE_INDICATOR,PRODUCT_STATUS,AVERAGE_DAILY_DEMAND_CASES,AVERAGE_VENDOR_ORDER_CYCLE_DAYS,AVERAGE_ORDER_CYCLE_DAYS,AVERAGE_ORDER_CYCLE_CASES,LEAD_TIME_TO_DISTANCE_RATIO
0,0,101,3960,2014-07-31,1295,1371,2014-07-26,0,36,0,...,0.71,6,5327,Y,0,1.48,7.0,7.020016,9.313729,0.021
1,2,101,5479,2014-09-02,1295,1371,2014-08-20,5,36,0,...,0.65,1,6135,Y,0,8.49,7.0,7.000000,52.623575,0.059


Index(['ID', 'DIVISION_NUMBER', 'PRODUCT_NUMBER', 'PURCHASE_ORDER_DUE_DATE',
       'COMPANY_VENDOR_NUMBER', 'SHIP_FROM_VENDOR', 'ORDER_DATE',
       'ORDER_DAY_OF_WEEK', 'PRODUCT_CLASSIFICATION', 'PURCHASE_ORDER_TYPE',
       'DISTANCE_IN_MILES', 'DIVISION_CODE', 'PURCHASE_FROM_VENDOR',
       'AVERAGE_PRODUCT_ORDER_QUANTITY_MARKET', 'ORDER_QUANTITY_DEVIATION',
       'TRANSIT_LEAD_TIME', 'PURCHASING_LEAD_TIME',
       'DAYS_BETWEEN_ORDER_AND_DUE_DATE', 'GIVEN_TIME_TO_LEAD_TIME_RATIO',
       'DUE_DATE_WEEKDAY', 'PRODUCT_MARKET', 'RESERVABLE_INDICATOR',
       'PRODUCT_STATUS', 'AVERAGE_DAILY_DEMAND_CASES',
       'AVERAGE_VENDOR_ORDER_CYCLE_DAYS', 'AVERAGE_ORDER_CYCLE_DAYS',
       'AVERAGE_ORDER_CYCLE_CASES', 'LEAD_TIME_TO_DISTANCE_RATIO'],
      dtype='object')
(20413, 28)


,ID,ON_TIME_AND_COMPLETE
0,0,1
1,2,0


(20413, 2)


In [5]:
# Examine the testing data
display(X_test.head(2))
print(X_test.shape)

,ID,DIVISION_NUMBER,PRODUCT_NUMBER,PURCHASE_ORDER_DUE_DATE,COMPANY_VENDOR_NUMBER,SHIP_FROM_VENDOR,ORDER_DATE,ORDER_DAY_OF_WEEK,PRODUCT_CLASSIFICATION,PURCHASE_ORDER_TYPE,...,GIVEN_TIME_TO_LEAD_TIME_RATIO,DUE_DATE_WEEKDAY,PRODUCT_MARKET,RESERVABLE_INDICATOR,PRODUCT_STATUS,AVERAGE_DAILY_DEMAND_CASES,AVERAGE_VENDOR_ORDER_CYCLE_DAYS,AVERAGE_ORDER_CYCLE_DAYS,AVERAGE_ORDER_CYCLE_CASES,LEAD_TIME_TO_DISTANCE_RATIO
0,11,102,3485,2014-10-20,1295,853,2014-10-18,0,36,1,...,1.0,3,5780,Y,0,12.8,7.000,7.000000,86.944667,0.046
1,19,100,5380,2014-10-28,44,45,2014-10-22,5,21,1,...,1.0,1,2414,Y,0,1.6,18.105,18.105882,21.264765,0.007


(10247, 28)


**Cleaning**

In [6]:
# Check for missing values
print(X_train.isna().sum().sort_values(ascending = False).head())
print(X_test.isna().sum().sort_values(ascending = False).head())

AVERAGE_VENDOR_ORDER_CYCLE_DAYS    339
AVERAGE_ORDER_CYCLE_DAYS           339
AVERAGE_ORDER_CYCLE_CASES          339
AVERAGE_DAILY_DEMAND_CASES          41
COMPANY_VENDOR_NUMBER                0
dtype: int64
AVERAGE_VENDOR_ORDER_CYCLE_DAYS    184
AVERAGE_ORDER_CYCLE_DAYS           184
AVERAGE_ORDER_CYCLE_CASES          184
AVERAGE_DAILY_DEMAND_CASES          13
COMPANY_VENDOR_NUMBER                0
dtype: int64


Four columns are missing data (for both the train and test datasets).

In [7]:
# Function to impute missing values for four columns (using median)
def clean_data(data):
    data['AVERAGE_VENDOR_ORDER_CYCLE_DAYS'] = data['AVERAGE_VENDOR_ORDER_CYCLE_DAYS'].fillna(data.AVERAGE_DAILY_DEMAND_CASES.median())
    data['AVERAGE_ORDER_CYCLE_DAYS'] = data['AVERAGE_ORDER_CYCLE_DAYS'].fillna(data.AVERAGE_DAILY_DEMAND_CASES.median())
    data['AVERAGE_ORDER_CYCLE_CASES'] = data['AVERAGE_ORDER_CYCLE_CASES'].fillna(data.AVERAGE_DAILY_DEMAND_CASES.median())
    data['AVERAGE_DAILY_DEMAND_CASES'] = data['AVERAGE_DAILY_DEMAND_CASES'].fillna(data.AVERAGE_DAILY_DEMAND_CASES.median())

# Apply to the datasets
clean_data(X_train)
clean_data(X_test)

In [8]:
# Function to remove columns
def remove(col):
    X_train.drop(col, axis = 1, inplace = True)
    X_test.drop(col, axis = 1, inplace = True)

# Apply on certain columns of no use
remove('ID')
remove('PURCHASE_ORDER_DUE_DATE')
remove('ORDER_DATE') # difference is same as 'DAYS_BETWEEN_ORDER_AND_DUE_DATE'
remove('RESERVABLE_INDICATOR') # only one value for all
remove('PRODUCT_STATUS') # only one value for all

**Categorical Data**

In [9]:
# Filter to ensure unique values are the same in train and test datasets
def filter_low_numbers(col, threshold):
    global X_train, X_test
    common_values = np.intersect1d(np.array(X_train_copy[col].value_counts().index), (np.array(X_test_copy[col].value_counts().index)))
    high_values = X_train_copy[col].value_counts()[X_train_copy[col].value_counts() > threshold].index
    good_values = np.intersect1d(common_values, high_values)
    X_train[col] = X_train[col].apply(lambda x: x if x in good_values else 'OTHER')
    X_test[col] = X_test[col].apply(lambda x: x if x in good_values else 'OTHER')

# Apply on categorical columns
filter_low_numbers('DIVISION_NUMBER', 1)
filter_low_numbers('PRODUCT_NUMBER', 1)
filter_low_numbers('COMPANY_VENDOR_NUMBER', 1)
filter_low_numbers('SHIP_FROM_VENDOR', 1)
filter_low_numbers('ORDER_DAY_OF_WEEK', 1)
filter_low_numbers('PRODUCT_CLASSIFICATION', 1)
filter_low_numbers('PURCHASE_ORDER_TYPE', 1)
filter_low_numbers('DIVISION_CODE', 1)
filter_low_numbers('DUE_DATE_WEEKDAY', 1)
filter_low_numbers('PRODUCT_MARKET', 0)

In [10]:
# Get dummies on categorical data
X_train_dummy = pd.get_dummies(X_train, drop_first = True)
X_test_dummy = pd.get_dummies(X_test, drop_first = True)

**Check**

In [11]:
X_train_dummy.shape

(20413, 6402)

In [12]:
X_test_dummy.shape

(10247, 6402)

The train and test datasets have the same number of columns; good to move on.

**Model**

In [16]:
# Find the best predictors
selector = SelectKBest(k = 100)
X_selected_train = selector.fit_transform(X_train_dummy, y_train['ON_TIME_AND_COMPLETE'])
X_selected_test = selector.transform(X_test_dummy)

# See predictors
#features = selector.get_feature_names_out()
#print(features)

In [17]:
# Polynomial Features
poly = PolynomialFeatures(2, include_bias = False)
X_train_poly = poly.fit_transform(X_selected_train)
X_test_poly = poly.transform(X_selected_test)

In [18]:
X_train_poly.shape

(20413, 5150)

In [19]:
# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

In [18]:
# Separate train and test
#X_train_validate, X_test_validate, y_train_validate, y_test_validate = train_test_split(pd.DataFrame(X_train_scaled), y_train['ON_TIME_AND_COMPLETE'], stratify = y_train['ON_TIME_AND_COMPLETE'],test_size = 0.25, random_state = 3)

In [20]:
# Create model
model = LogisticRegression(solver = 'saga', penalty = 'l1', C = 1, n_jobs = -1)
model.fit(X_train_scaled, y_train['ON_TIME_AND_COMPLETE'])

/Users/gracefan/anaconda3/envs/stat303_conda_env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=1, n_jobs=-1, penalty='l1', solver='saga')

In [21]:
# Find probabilities
probabilities = model.predict_proba(X_train_scaled)[:,1]

# Create possible thresholds
thresholds = np.linspace(0, 1, 100)

# Find all the accuracies for each threshold
accuracies = [accuracy_score(y_train['ON_TIME_AND_COMPLETE'], (probabilities >= t).astype(int)) for t in thresholds]

# Find the best threshold
threshold = thresholds[np.argmax(accuracies)]
print(threshold)

0.5050505050505051


In [22]:
# Look at train validate metrics
probabilities = model.predict_proba(X_train_scaled)[:,1]
predictions = (probabilities >= threshold).astype(int)
print('Accuracy: ', accuracy_score(y_train['ON_TIME_AND_COMPLETE'], predictions))
print('Precision: ', precision_score(y_train['ON_TIME_AND_COMPLETE'], predictions))
print('Recall: ', recall_score(y_train['ON_TIME_AND_COMPLETE'], predictions))
print('F1: ', f1_score(y_train['ON_TIME_AND_COMPLETE'], predictions))

Accuracy:  0.8101210013226865
Precision:  0.8273790365840575
Recall:  0.7728229281491131
F1:  0.7991709844559586


In [23]:
# Look at the predictions
predictions

array([1, 0, 0, ..., 1, 0, 0])

**CSV File**

In [24]:
# Prepare the dataset
probabilities = model.predict_proba(X_test_scaled)[:,1]
predictions = (probabilities >= 0.5).astype(int)
X_test2 = pd.read_csv('../Datasets/public_private_X.csv')
X_test2['ON_TIME_AND_IS_COMPLETE'] = predictions
final = X_test2[['ID', 'ON_TIME_AND_IS_COMPLETE']]
display(final.tail())
display(final.shape)

,ID,ON_TIME_AND_IS_COMPLETE
10242,30641,1
10243,30642,1
10244,30654,1
10245,30657,0
10246,30659,1


(10247, 2)

In [25]:
# Create the CSV file
#final.to_csv('Predictions11.csv', index = False)

In [26]:
# Check the file
#predictions_csv = pd.read_csv('Predictions11.csv')
#display(predictions_csv.tail())
#print(predictions_csv.shape)

,ID,ON_TIME_AND_IS_COMPLETE
10242,30641,1
10243,30642,1
10244,30654,1
10245,30657,0
10246,30659,1


(10247, 2)
